## Importing modules and Loading dataset.
This section contains importing the important python modules. Also, the dataset to be used, in this case the   



In [ ]:
# importing python module.
import pandas as pd
from lightgbm.sklearn import LGBMClassifier
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# loading dataset with pandas
dia = pd.read_csv("./dataset/diabetes.csv")

dia.head()

In [ ]:
# wrangling datasets with pandas
dia.info()

In [ ]:
dia.describe()

## Wrangling dataset.

In [ ]:
dia.chol_hdl_ratio = round(dia.cholesterol / dia.hdl_chol,2)

In [ ]:
dia.head()

In [ ]:
dia.waist_hip_ratio= round(dia.waist/dia.hip,2)

In [ ]:
dia.head()

In [ ]:
# creating feature related to weight and height

dia["weight_height"] = round(dia.weight/dia.height, 2)

dia.head()

In [ ]:
dia.bmi = pd.to_numeric(dia.bmi.str.replace(",","."))

In [ ]:
dia.head()

In [ ]:
dia.weight.describe()

In [ ]:
dia.isnull().sum()

In [ ]:
dia.rename(columns={"weight":"height", "height":"weight", "weight_height":"height_weight"}, inplace=True)

In [ ]:
dia.head()

In [ ]:
s= (dia.dtypes == "object")
obj_col= list(s[s].index)

In [ ]:
obj_col

In [ ]:
orde = OrdinalEncoder()
dia[obj_col] = orde.fit_transform(dia[obj_col])

In [ ]:
dia.head()

In [ ]:
X = dia.drop(["patient_number", "diabetes"], axis= 1)
y= dia.diabetes

In [ ]:
scale = RobustScaler()
scaled_x = scale.fit_transform(X, y=y)

In [ ]:
scaled_x

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size= 0.2, random_state=42)

In [ ]:
split = StratifiedShuffleSplit(n_splits=4, random_state=42 )

for train_index, test_index in split.split(scaled_x, y):
    strat_X, strat_test = scaled_x[train_index], scaled_x[test_index]
    strat_y, strat_ytest = y[train_index], y[test_index]

In [ ]:
X_train

In [ ]:
lgbm_model = LGBMClassifier(n_estimators=200, max_depth=-2, random_state=42)

In [ ]:
lgbm_model.fit(X_train, y_train)

In [ ]:
y_pred=lgbm_model.predict(X_test)

In [ ]:
f1_score(y_pred, y_test)

In [ ]:
xg_model= XGBClassifier(n_estimators=200, max_depth=4, scale_pos_weight=5.5)

In [ ]:
xg_model.fit(X_train, y_train)

In [ ]:
xg_pred = xg_model.predict(X_test)

In [ ]:
f1_score(y_test, xg_pred)

## Stratified Shuffle Test

In [ ]:
lgbm_strat = LGBMClassifier(n_estimators=200, learning_rate=0.0099, max_depth=-2, )

In [ ]:
lgbm_strat.fit(strat_X, strat_y)

In [ ]:
strat_pred = lgbm_strat.predict(strat_test)

In [ ]:
f1_score(strat_pred, strat_ytest)

In [ ]:
var = [200, 80, 67, 2.99, 20, 1.0, 60, 160, 24.6, 120, 60, 32, 38, 0.84, 2.67]
import numpy as np

def predict(var_name):
    pred = [var_name]
    np_pred = np.array(pred)
    score = lgbm_strat.predict(np_pred)
    return score

In [ ]:
predict(var)

In [ ]:
dia.diabetes.value_counts()

In [ ]:
330/60

In [ ]:
xgb= XGBClassifier(max_depth=7, n_estimators=1000, scale_pos_weight=5.5)

In [ ]:
xgb.fit(strat_X, strat_y)

In [ ]:
y=xgb.predict(strat_test)

In [ ]:
f1_score(y, strat_ytest)

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect("diabetes.db")
c = conn.cursor()

In [ ]:
col= tuple(dia.columns)
col

In [ ]:
conn.commit()

In [ ]:
dia.to_sql(name="diabetes.db", con=conn, if_exists= "replace", index=False)

In [ ]:
for index in split.split(scaled_x, y):
    X_total, y_total = scaled_x[index], y[index]

In [ ]:
y